<a href="https://colab.research.google.com/github/chrisliii/FINE-452/blob/main/Fama_French.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import statsmodels.api as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
stock_name = "^GSPC"
ticker = yf.Ticker(stock_name)
#ticker.info
# monthly data
df_ticker_monthly_return = ticker.history(period="max", interval='1mo')
# monthly return
df_ticker_monthly_return['MonthlyReturn'] = 100*( (df_ticker_monthly_return['Close'].shift(-1)).div(df_ticker_monthly_return['Close']) - 1)
df_ticker_monthly_return['MonthlyReturn'] = df_ticker_monthly_return['MonthlyReturn'].shift(1)
df_ticker_monthly_return = df_ticker_monthly_return.dropna()
#date to string
df_ticker_monthly_return = df_ticker_monthly_return.reset_index()
df_ticker_monthly_return['Date'] = df_ticker_monthly_return['Date'].apply(lambda x: x.strftime('%Y%m'))

df_ticker_monthly_return.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MonthlyReturn
0,192801,17.76,17.76,17.26,17.57,0,0,0,-0.509626
1,192802,17.53,17.63,16.95,17.26,0,0,0,-1.764371
2,192803,17.30,19.28,17.30,19.28,0,0,0,11.703360
3,192804,18.91,19.75,18.91,19.75,0,0,0,2.437759
4,192805,19.78,20.44,19.36,20.00,0,0,0,1.265823


In [12]:
stock_name = "DELL"
ticker = yf.Ticker(stock_name)
#ticker.info
# monthly data
df_ticker_monthly_return = ticker.history(period="max", interval='1mo')
# process Dividends
df_ticker_monthly_return['Dividends'] = df_ticker_monthly_return['Dividends'].shift(1)
df_ticker_monthly_return = df_ticker_monthly_return.dropna()
# monthly return
df_ticker_monthly_return['MonthlyReturn'] = 100*( (df_ticker_monthly_return['Close'].shift(-1)+df_ticker_monthly_return['Dividends'].shift(-1)).div(df_ticker_monthly_return['Close']) - 1)
df_ticker_monthly_return['MonthlyReturn'] = df_ticker_monthly_return['MonthlyReturn'].shift(1)
df_ticker_monthly_return = df_ticker_monthly_return.dropna()
#date to string
df_ticker_monthly_return = df_ticker_monthly_return.reset_index()
df_ticker_monthly_return['Date'] = df_ticker_monthly_return['Date'].apply(lambda x: x.strftime('%Y%m'))

df_ticker_monthly_return.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MonthlyReturn
0,200909,32.021257,36.097375,31.192473,34.613094,5235600.0,0.0,0,5.852513
1,200910,33.859653,39.585809,32.714422,35.547363,6638200.0,0.0,0,2.699178
2,200911,36.089851,39.593352,34.869273,37.890575,9535400.0,0.0,0,6.591803
3,200912,38.425517,40.452274,37.860436,39.201561,4094100.0,0.0,0,3.459925
4,201001,40.173503,40.459811,35.743266,37.513855,6414800.0,0.0,0,-4.305201


In [13]:
df_FF_Research = pd.read_csv('F-F_Research_Data_Factors.CSV', encoding="utf-8", skipinitialspace=True)
df_FF_Research.columns = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']
df_FF_Research = df_FF_Research.dropna()
df_FF_Research['Mkt-RF'] = df_FF_Research['Mkt-RF'].astype(float)
df_FF_Research['SMB'] = df_FF_Research['SMB'].astype(float)
df_FF_Research['HML'] = df_FF_Research['HML'].astype(float)

df_FF_Research.head()

,Date,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.30,-2.87,0.22
1,192608,2.64,-1.40,4.19,0.25
2,192609,0.36,-1.32,0.01,0.23
3,192610,-3.24,0.04,0.51,0.32
4,192611,2.53,-0.20,-0.35,0.31


In [22]:
# Before merger
df_ticker_FF = pd.merge(df_ticker_monthly_return, df_FF_Research, on='Date')
df_ticker_FF['Date'] = df_ticker_FF['Date'].astype(int)
df_ticker_FF = df_ticker_FF[(df_ticker_FF.Date >= 201009) & 
                            (df_ticker_FF.Date <= 201509)]

model = sm.OLS(df_ticker_FF['MonthlyReturn'], sm.add_constant(
    df_ticker_FF[['Mkt-RF', 'SMB', 'HML']].values))
result = model.fit()
print(stock_name + '\n')
print (result.params)
print(result.summary())

BUD

const    0.429189
x1       1.016587
x2      -0.338834
x3      -0.529946
dtype: float64
                            OLS Regression Results                            
Dep. Variable:          MonthlyReturn   R-squared:                       0.393
Model:                            OLS   Adj. R-squared:                  0.361
Method:                 Least Squares   F-statistic:                     12.31
Date:                Wed, 28 Oct 2020   Prob (F-statistic):           2.57e-06
Time:                        04:59:11   Log-Likelihood:                -175.71
No. Observations:                  61   AIC:                             359.4
Df Residuals:                      57   BIC:                             367.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

In [23]:
# After merger
df_ticker_FF = pd.merge(df_ticker_monthly_return, df_FF_Research, on='Date')
df_ticker_FF['Date'] = df_ticker_FF['Date'].astype(int)
df_ticker_FF = df_ticker_FF[(df_ticker_FF.Date >= 201511) & 
                            (df_ticker_FF.Date <= 202011)]

model = sm.OLS(df_ticker_FF['MonthlyReturn'], sm.add_constant(
    df_ticker_FF[['Mkt-RF', 'SMB', 'HML']].values))
result = model.fit()
print(stock_name + '\n')
print (result.params)
print(result.summary())

BUD

const   -1.665961
x1       1.312990
x2      -0.389646
x3       0.122467
dtype: float64
                            OLS Regression Results                            
Dep. Variable:          MonthlyReturn   R-squared:                       0.412
Model:                            OLS   Adj. R-squared:                  0.379
Method:                 Least Squares   F-statistic:                     12.39
Date:                Wed, 28 Oct 2020   Prob (F-statistic):           2.97e-06
Time:                        04:59:12   Log-Likelihood:                -188.25
No. Observations:                  57   AIC:                             384.5
Df Residuals:                      53   BIC:                             392.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

In [7]:
df_ticker_monthly_return.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,MonthlyReturn
405,202007,434.799988,470.609985,416.290009,444.320007,54649300.0,0.0,0.0,2.069792
406,202008,449.839996,533.700012,431.589996,513.390015,54517800.0,0.0,0.0,15.545104
407,202009,515.000000,536.880005,452.519989,490.429993,84770900.0,0.0,0.0,-4.472238
408,202010,497.429993,519.599976,470.130005,475.200012,38118500.0,0.0,0.0,-3.105434
409,202010,479.779999,481.665009,473.470001,478.559998,1896459.0,0.0,0.0,0.707068
